In [3]:
import toml
from tqdm import tqdm

In [4]:
# Get data and inference ids
data_ids = ["exponential_fd_log_simulation",
            "greenshields_fd_log_simulation",
            "daganzos_fd_log_simulation",
            "delcastillos_fd_log_simulation",
            "greenbergs_fd_log_simulation",
            "underwoods_fd_log_simulation",
            "northwesterns_fd_log_simulation",
            "newells_fd_log_simulation",
            "wangs_fd_log_simulation",
            "smulders_fd_log_simulation"]
inference_ids = ["grwmh_exponential_model_exponential_sim_learn_noise",
                  "grwmh_greenshields_model_greenshields_sim_learn_noise",
                  "grwmh_daganzos_model_daganzos_sim_learn_noise",
                  "grwmh_delcastillos_model_delcastillos_sim_learn_noise",
                  "grwmh_greenbergs_model_greenbergs_sim_learn_noise",
                  "grwmh_underwoods_model_underwoods_sim_learn_noise",
                  "grwmh_northwesterns_model_northwesterns_sim_learn_noise",
                  "grwmh_newells_model_newells_sim_learn_noise",
                  "grwmh_wangs_model_wangs_sim_learn_noise",
                  "grwmh_smulders_model_smulders_sim_learn_noise"]

## Create simulation parameters for multiple dataset sizes

In [ ]:
for data_id in tqdm(data_ids):
    
    # Import toml file
    